In [0]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
import org.apache.spark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import Row
from pyspark.context import SparkContext
import numpy as np

import collections

from collections import Counter

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit


- Student name: Ardian Lagman
- Student number: 1003060135

In [0]:
#pip install pyspark

In [0]:
spark = SparkSession.builder.appName("APS1052_HW2_SparkSession").getOrCreate()

#Part A

## 1. Count the odd and even numbers using file 'integer.txt' download it from Quercus. show your code and output

In [0]:
integerRDD = spark.sparkContext.textFile("/FileStore/tables/integer.txt")

In [0]:
raw_RDD = integerRDD.collect()

In [0]:
type(raw_RDD)

Out[6]: list

In [0]:
type(raw_RDD[0])

Out[7]: str

In [0]:
integer_array = np.array(raw_RDD)

In [0]:
odd_counter = 0
even_counter = 0

for integer in integer_array:
    if int(integer) % 2 == 0:
        even_counter += 1
    else:
        odd_counter += 1

In [0]:
odd_counter

Out[10]: 496

In [0]:
even_counter

Out[11]: 514

## 2. Calculate the salary sum per department using file 'salary.txt' download it from Quercus. Show department name and salary sum. Show your code and output

In [0]:
salaryRDD = spark.sparkContext.textFile("/FileStore/tables/salary.txt")

In [0]:
#salaryRDD.collect()

In [0]:
splitSalaryRDD = [item.split(" ") for item in salaryRDD.collect()]

In [0]:
#splitSalaryRDD

In [0]:
salarySumDict = {}

for item in splitSalaryRDD:
    jobTitle = item[0]
    jobSalary = int(item[1])
    
    if jobTitle in salarySumDict:
        salarySumDict[jobTitle] += jobSalary
    else:
        salarySumDict[jobTitle] = jobSalary
    

In [0]:
salarySumDict

Out[17]: {'Sales': 3488491,
 'Research': 3328284,
 'Developer': 3221394,
 'QA': 3360624,
 'Marketing': 3158450}

## 3. Implement MapReduce using Pyspark on file 'shakespeare.txt' download it from Quercus. Show how many times these particular words appear in the document: Shakespeare, When, Lord, Library, GUTENBERG, WILLIAM, COLLEGE, and WORLD

In [0]:
shakespeareRDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_2.txt")

In [0]:
#shakespeareRDD.collect()

In [0]:
stringList = ["Shakespeare", "When", "Lord", "Library", "GUTENBERG", "WILLIAM", "COLLEGE", "WORLD"]

In [0]:
counts = shakespeareRDD.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

In [0]:
for item in counts.collect():
    if item[0] in stringList:
        print(item)

('Shakespeare', 22)
('GUTENBERG', 99)
('WILLIAM', 115)
('WORLD', 98)
('COLLEGE', 98)
('When', 393)
('Lord', 341)
('Library', 2)


## 4. Calculate the top 15 and bottom 15 words using file 'shakespeare.txt' download it from Quercus. Show 15 words with most count and 15 words with least count. You can limit by 15 in ascending and descending order of yoru count

In [0]:
rawShakespeareRDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_2.txt")

In [0]:
#rawShakespeareRDD.collect()

In [0]:
shakespeareArray = [item.split(" ") for item in rawShakespeareRDD.collect()]

In [0]:
#shakespeareArray

In [0]:
counts = Counter()

for line in shakespeareArray:
    for word in line:
        counts[word] += 1

In [0]:
most_common = counts.most_common()[0:15]

In [0]:
most_common

Out[80]: [('', 231583),
 ('the', 11397),
 ('and', 8777),
 ('I', 8556),
 ('of', 7873),
 ('to', 7421),
 ('a', 5672),
 ('my', 4913),
 ('in', 4600),
 ('you', 4060),
 ('And', 3547),
 ('that', 3522),
 ('is', 3481),
 ('his', 3226),
 ('with', 3175)]

In [0]:
least_common = counts.most_common()[-15:]

In [0]:
least_common

Out[31]: [('gives,', 1),
 ('seventy-five', 1),
 ('drachmas.', 1),
 ('arbors,', 1),
 ('new-planted', 1),
 ('orchards,', 1),
 ('Tiber;', 1),
 ('forever-', 1),
 ('recreate', 1),
 ('houses.', 1),
 ('benches.', 1),
 ('Mischief,', 1),
 ('Octavius.', 1),
 ('Cinna', 1),
 ('poet.', 1)]

# Part B

## 1. Describe your data. Calculate top 10 movies with the highest ratings and top 10 uers who provide highest ratings

In [0]:
movies = spark.read.csv('/FileStore/tables/movies.csv', sep = ",", inferSchema = True, header = True)

In [0]:
df_movies = movies.toPandas()

In [0]:
df_movies.shape

Out[34]: (1501, 3)

In [0]:
df_movies.head()

,movieId,rating,userId
0,2,3,0
1,3,1,0
2,5,2,0
3,9,4,0
4,11,1,0


In [0]:
df_movies.sort_values(['rating'], ascending = False).head(10)

,movieId,rating,userId
143,93,5,2
1145,75,5,22
561,18,5,11
566,23,5,11
568,27,5,11
569,30,5,11
570,32,5,11
1131,51,5,22
581,48,5,11
1411,12,5,28


Comments
- top 10 movies are with the id's as shown

In [0]:
df_movies.sort_values(['rating', 'userId'], ascending = False).userId.value_counts()

Out[37]: 14    57
6     57
22    56
11    56
4     55
12    55
7     54
9     53
24    52
23    52
18    52
28    50
19    49
8     49
5     49
26    49
1     49
0     49
21    48
15    48
13    48
3     48
20    47
17    46
25    46
27    46
2     46
29    46
16    45
10    44
Name: userId, dtype: int64

top ten users are as follows:
- 6
- 22
- 11
- 4
- 12
- 7
- 9
- 24
- 23
- 18

Notes
- need to convert the dataframe where the individual row indexes are the movie ID's 
- Each row needs to be specific ratings
- the columns are specific users
- My plan is to initialize an empty numpy array and start filling out each individual row (movie) with each individual viewer rating (column)

In [0]:
df_movies['movieId'].unique()

Out[38]: array([ 2,  3,  5,  9, 11, 12, 15, 17, 19, 21, 23, 26, 27, 28, 29, 30, 31,
       34, 37, 41, 44, 45, 46, 47, 48, 50, 51, 54, 55, 59, 61, 64, 67, 68,
       69, 71, 72, 77, 79, 83, 87, 89, 91, 92, 94, 95, 96, 98, 99,  4,  6,
       13, 14, 16, 33, 36, 40, 43, 56, 57, 58, 60, 62, 63, 70, 73, 74, 76,
       78, 81, 82, 85, 86, 88, 93, 97,  8, 10, 18, 22, 35, 38, 39, 52, 65,
       66, 80, 90,  0,  1,  7, 24, 84, 20, 49, 42, 25, 53, 75, 32],
      dtype=int32)

In [0]:
len(df_movies['movieId'].unique())

Out[39]: 100

In [0]:
df_movies['userId'].unique()

Out[40]: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)

In [0]:
df_movies['rating'].unique()

Out[41]: array([3, 1, 2, 4, 5], dtype=int32)

In [0]:
len(df_movies['userId'].unique())

Out[42]: 30

In [0]:
df_movies.loc[df_movies['movieId'] == 1]

,movieId,rating,userId
145,1,1,3
192,1,1,4
248,1,1,5
297,1,1,6
353,1,1,7
712,1,1,14
770,1,4,15
908,1,1,18
961,1,1,19
1010,1,1,20


## 2. Split the dataset into train and test. Try 2 different combinations

In [0]:
#Splititng data to 80/20 and 70/30
train80, test20 = movies.randomSplit([0.8, 0.2])
train70, test30 = movies.randomSplit([0.7, 0.3])

In [0]:
als70 = ALS(userCol="userId", itemCol = 'movieId', ratingCol = 'rating', coldStartStrategy = "drop")

In [0]:
als80 = ALS(userCol="userId", itemCol = 'movieId', ratingCol = 'rating', coldStartStrategy = "drop")

## 3. Explain MSE, RMSE, and MAE

- MSE is using the Mean Squared Error, where where we find the average of the square of the errors between the prediction and the true label
- RMSE is the Root Mean Squared Error, where it is like MSE except we also find the square root of it all
- MAE is the mean absolute error, where we find the absolute error of all the predictions and true labels and find the mean of that value

In [0]:
evalMSE = RegressionEvaluator(
    metricName = 'mse',
    labelCol = "rating",
    predictionCol = "prediction"
)

In [0]:
evalRMSE = RegressionEvaluator(
    metricName = 'rmse',
    labelCol = 'rating',
    predictionCol = 'prediction'
)

In [0]:
evalMAE = RegressionEvaluator(
    metricName = 'mae',
    labelCol = 'rating',
    predictionCol = 'prediction'
)

In [0]:
model80 = als80.fit(train80)

In [0]:
model80_predict = model80.transform(test20)

In [0]:
model80_MSE = evalMSE.evaluate(model80_predict)
print("MSE: " + str(model80_MSE))

MSE: 0.9070580080987767


In [0]:
model80_RMSE = evalRMSE.evaluate(model80_predict)
print("RMSE: " + str(model80_RMSE))

RMSE: 0.952395930324556


In [0]:
model80_MAE = evalMAE.evaluate(model80_predict)
print("MAE: " + str(model80_MAE))

MAE: 0.6794477630495618


In [0]:
model70 = als70.fit(train70)

In [0]:
model70_predict = model70.transform(test30)

In [0]:
model70_MSE = evalMSE.evaluate(model70_predict)
print("MSE: " + str(model70_MSE))

MSE: 1.1996502468349568


In [0]:
model70_RMSE = evalRMSE.evaluate(model70_predict)
print("RMSE: " + str(model70_RMSE))

RMSE: 1.0952854636280702


In [0]:
model70_MAE = evalMAE.evaluate(model70_predict)
print("MAE: " + str(model70_MAE))

MAE: 0.767729618650829


Comments
- it appears that MSE and RMSE may appear to give higher error values compared to MAE
- for my model, I believe that higher error value can help force the model to learn better
- Given that Mean Squared Error, by definition, focuses on the squares of the error, I believe this will help the model learn better because it will be more aware of bigger errors

# 4. Now tune your parameters

In [0]:
parameters80 = ParamGridBuilder().addGrid(als80.rank, [10, 20]).addGrid(als80.maxIter, [10, 15]).addGrid(als80.regParam, [1.0, 1.5]).build()

In [0]:
parameters70 = ParamGridBuilder().addGrid(als70.rank, [10, 15]).addGrid(als70.maxIter, [10, 15]).addGrid(als70.regParam, [1.0, 1.5]).build()

In [0]:
train_vs_80 = TrainValidationSplit(
    estimator = als80,
    estimatorParamMaps = parameters80,
    evaluator = evalMSE,
    trainRatio = 0.80
)

In [0]:
model_vs_80 = train_vs_80.fit(train80)

/databricks/spark/python/pyspark/ml/util.py:886: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
model_vs_80_predict = model_vs_80.transform(test20)

In [0]:
model_vs_80_predict.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     11|     1|     0|0.44951472|
|     37|     1|     0|0.39636517|
|     41|     2|     0|0.62977797|
|     45|     2|     0|0.41764933|
|     67|     1|     0| 0.5325426|
|     72|     1|     0| 0.6332112|
|     79|     1|     0| 0.6211853|
|     91|     3|     0| 0.5012382|
|     95|     2|     0| 0.6103914|
|      6|     1|     1| 0.5861582|
|     36|     2|     1|0.61241794|
|     37|     1|     1|0.45349327|
|     40|     1|     1| 0.6260021|
|     58|     1|     1|0.50948286|
|     77|     3|     1|0.66621214|
|     85|     3|     1| 0.7333488|
|      8|     5|     2| 0.8475784|
|      9|     1|     2|0.64084363|
|     10|     1|     2|0.66800636|
|     18|     2|     2| 1.2036242|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
model_vs_80_mse = evalMSE.evaluate(model_vs_80_predict)
print("MSE: " + str(model_vs_80_mse))

MSE: 2.3326100531561873


In [0]:
train_vs_70 = TrainValidationSplit(
    estimator = als80,
    estimatorParamMaps = parameters70,
    evaluator = evalMSE,
    trainRatio = 0.70
)

In [0]:
model_vs_70= train_vs_70.fit(train70)

In [0]:
model_vs_70_predict = model_vs_70.transform(test30)

In [0]:
model_vs_70_predict.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      2|     3|     0| 2.3509216|
|      3|     1|     0| 0.7410733|
|      9|     4|     0| 1.1156859|
|     23|     1|     0| 1.4717163|
|     34|     1|     0| 1.2933617|
|     46|     1|     0|0.82319516|
|     50|     1|     0|0.70882714|
|     54|     1|     0|0.79582196|
|     64|     1|     0| 1.6857302|
|     77|     2|     0| 1.2854252|
|     79|     1|     0| 0.6483449|
|     89|     2|     0| 1.0496104|
|     95|     2|     0| 1.3385909|
|      6|     1|     1| 1.3209406|
|      9|     3|     1| 1.0718348|
|     12|     1|     1|  1.747317|
|     16|     1|     1| 1.3057064|
|     27|     1|     1|0.76152515|
|     37|     1|     1|0.97160965|
|     40|     1|     1| 1.3709108|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
model_vs_70_mse = evalMSE.evaluate(model_vs_70_predict)
print("MSE: " + str(model_vs_70_mse))

MSE: 1.1996502468349568


Comments
- given the lwoer MSE, I'll take the cross/val with model70

## Calculate top 10 movie recommendations for user id 10 and user id 14

In [0]:
model70_fit = als70.fit(train70)

In [0]:
user_id = [10,14]

user_recs = model70_fit.recommendForAllUsers(10)

In [0]:
user_recs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 3.9619672},...|
|    10|[{92, 3.4845748},...|
|     0|[{92, 3.0957658},...|
|     1|[{62, 2.944469}, ...|
|    21|[{53, 4.258135}, ...|
|    11|[{30, 4.729922}, ...|
|    12|[{55, 4.486953}, ...|
|    22|[{51, 4.6741242},...|
|     2|[{8, 4.508601}, {...|
|    13|[{93, 2.9906147},...|
|     3|[{74, 4.1736736},...|
|    23|[{55, 4.721018}, ...|
|     4|[{52, 3.254}, {93...|
|    24|[{52, 4.305026}, ...|
|    14|[{52, 4.658917}, ...|
|     5|[{55, 4.317712}, ...|
|    15|[{1, 2.989226}, {...|
|    25|[{83, 3.1389313},...|
|    26|[{22, 4.7505665},...|
|     6|[{25, 3.7450128},...|
+------+--------------------+
only showing top 20 rows



In [0]:
user10 = user_recs.filter(pyspark.sql.functions.col('userID') == 10)

In [0]:
user10.show(truncate = False)

+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                    |
+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10    |[{92, 3.4845748}, {2, 3.40339}, {40, 3.0196204}, {49, 2.843896}, {25, 2.7876694}, {12, 2.6423554}, {82, 2.533945}, {42, 2.5034366}, {4, 2.4133816}, {0, 2.2084932}]|
+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+



comments
- User 10 recommendations are movies 81, 49, 92, 12, 89, 2, 82, 91, 46, and 41

In [0]:
user14 = user_recs.filter(pyspark.sql.functions.col('userID') == 14)
user14.show(truncate = False)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                       |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|14    |[{52, 4.658917}, {76, 4.2332625}, {63, 4.222856}, {25, 3.8376257}, {72, 3.6222825}, {85, 3.4765825}, {62, 3.4649746}, {58, 3.450089}, {96, 3.217226}, {53, 2.9577112}]|
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+



Comments
- User 14 recommendations are movies 52, 29, 63, 76, 53, 85, 72, 25, 62, and 41

Other comments
- I understand that I did not have the most diverse parameter grid, but in the interest of time, I had to cut it down to a small size like this
- If I had the time, I would've done parameter grids such as [5, 10, 15, 20] or [0.5, 1.0, 1.5, 2.0]